In [2]:
import pandas as pd 
from sqlalchemy import and_
from models import Player, TradPlayerStats, AdvPlayerStats, Game
from data_manager import DataManager
dm = DataManager()
session = dm.get_session()
player_name = "Nikola Jokic"
player = session.query(Player).filter(Player.name==player_name).all()[0]
player_id = player.id

stats = session.query(TradPlayerStats).join(TradPlayerStats.game).filter(TradPlayerStats.player_id == player_id).all()
stats = dm.convert_to_df(stats)
print(stats.columns)


Index(['fgm', 'ft_pct', 'pf', 'fga', 'oreb', 'pts', 'fg_pct', 'dreb',
       'plus_minus', 'game_id', 'fg3m', 'reb', 'id', 'fg3a', 'ast',
       'player_id', 'fg3_pct', 'stl', 'start_position', 'ftm', 'blk',
       'minutes', 'fta', 'to'],
      dtype='object')


In [7]:

# Query to join four tables with correct join conditions and multiple filters
def get_and_save_player_data(player_id, player_name):
    data = session.query(
        Player,
        TradPlayerStats,
        AdvPlayerStats,
        Game
    ).join(Game, TradPlayerStats.game_id == Game.id)\
    .join(Player, TradPlayerStats.player_id == Player.id)\
    .join(AdvPlayerStats, and_(TradPlayerStats.game_id == AdvPlayerStats.game_id, 
                                TradPlayerStats.player_id == AdvPlayerStats.player_id))\
    .filter(
        (TradPlayerStats.player_id == player_id)# &
        #  (Game.season_type == "Playoffs")
    ).all()

    # Convert the query result to a DataFrame
    data_list = []
    for player, trad_stats, adv_stats, game in data:
        row = {
            'player_name': player.name,
            'player_position': player.position,
            'minutes': trad_stats.minutes,
            'points': trad_stats.pts,
            'rebounds': trad_stats.reb,
            'assists': trad_stats.ast,
            'efg': adv_stats.efg_pct,
            'fg3a': trad_stats.fg3a,
            'fg3m': trad_stats.fg3m,
            'fg3_pct': trad_stats.fg3_pct,
            'fga': trad_stats.fga,
            'fgm': trad_stats.fgm,
            'fta': trad_stats.fta,
            'ft_pct': trad_stats.ft_pct, 
            'date': game.date,

        }
        data_list.append(row)

    data_df = pd.DataFrame(data_list)

    data_df.to_csv("data_pile/{player_name}.csv")
    return data_df


     player_name player_position  minutes  points  rebounds  assists    efg  \
0   Nikola Jokic               C     41.0      25        20        9  0.595   
1   Nikola Jokic               C     41.0      33        14       14  0.550   
2   Nikola Jokic               C     38.0      24        15        9  0.692   
3   Nikola Jokic               C     41.0      27        20       10  0.625   
4   Nikola Jokic               C     39.0      32        12        7  0.674   
..           ...             ...      ...     ...       ...      ...    ...   
83  Nikola Jokic               C     42.0      38        19        8  0.500   
84  Nikola Jokic               C     38.0      36        21       11  0.538   
85  Nikola Jokic               C     40.0      36        17       10  0.646   
86  Nikola Jokic               C     37.0      22        15       10  0.313   
87  Nikola Jokic               C     35.0      32        16        9  0.391   

    fg3a  fg3m  fg3_pct  fga  fgm  fta  ft_pct     

In [8]:
data_df = pd.DataFrame(data_list)
data_df.to_csv('data.csv')